### Quickstart: Compare runs, choose a model, and deploy it to a REST API

In this quickstart, you will:

- Run a hyperparameter sweep on a training script

- Compare the results of the runs in the MLflow UI

- Choose the best run and register it as a model

- Deploy the model to a REST API

- Build a container image suitable for deployment to a cloud platform


In [2]:
import numpy as np
import pandas as pd
import mlflow
import keras
from hyperopt import STATUS_OK,Trials,fmin,hp,tpe
from sklearn.metrics import mean_squared_error
from mlflow.models import infer_signature
import mlflow.tensorflow
from sklearn.model_selection import train_test_split



In [3]:
## Load the Datset
data=pd.read_csv("https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/refs/heads/master/winequality-white.csv")

# Splite data into train,validatio, test sets

train,test=train_test_split(data,test_size=0.2,random_state=42)



In [4]:
train[['quality']].values.ravel()

array([6, 5, 6, ..., 6, 6, 8])

In [5]:
X_train = train.drop('quality',axis=1).values
y_train=train[['quality']].values.ravel()
X_train,X_valid,y_train,y_valid = train_test_split(X_train,y_train,test_size=0.2,random_state=42)

#test Dataset
X_test = test.drop('quality',axis=1).values
y_test = test[['quality']].values.ravel()

#mlflow Signature infer
sign = infer_signature(X_train,y_train)

In [6]:
def train_model(params,epochs,x_train,y_train,x_valid,y_valid,x_test,y_test):
    #Get mean and varience for normalization
    mean=np.mean(x_train,axis=0)
    var=np.var(x_train,axis=0)

    #Define Model Architecture
    model= keras.Sequential([
        keras.Input([x_train.shape[1]]),
        keras.layers.Normalization(mean=mean,variance=var),
        keras.layers.Dense(100,activation='relu'),
        keras.layers.Dense(1),
    ])
    #Compile Model
    model.compile(loss="mean_squared_error",optimizer=keras.optimizers.SGD(learning_rate=params['lr'],momentum=params['momentum'])
                  ,metrics=[keras.metrics.RootMeanSquaredError()])
    #Train the ANN Model
    with mlflow.start_run(nested=True):
        model.fit(x_train,y_train,validation_data=(x_valid,y_valid),
                  epochs=epochs,batch_size=64)
        ## Evaluate the model
        eval=model.evaluate(x_valid,y_valid,batch_size=64)
        eval_rmse=eval[1]

        ##Log the Params & results
        mlflow.log_params(params)
        mlflow.log_metric('eval_rmse',eval_rmse)
        ## Log the model
        mlflow.tensorflow.log_model(model,'model',signature=sign)
        return {'loss':eval_rmse,'status':STATUS_OK,'model':model}

In [7]:
def objective(params):
    #Mlflow will track the parameters and results for each run
    result=train_model(params,
                       epochs=3,
                       x_train=X_train,
                       y_train=y_train,
                       x_valid=X_valid,
                       y_valid=y_valid,
                       x_test=X_test,
                       y_test=y_test)
    return result


In [14]:
space = {
    'lr':hp.loguniform('lr',np.log(1e-5),np.log(1e-1)),
    'momentum':hp.uniform('momentum',0.0,1.0)
    
}

In [ ]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("wine-quality")

with mlflow.start_run():
    #condact the hyperparameter search using hyperopt
    #trials is a DB to store the each parameter value and the result 
    trials=Trials()
    best=fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials
    )
     # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=sign)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")

  0%|          | 0/4 [00:00<?, ?trial/s, best loss=?]

2025-05-31 01:07:43.332080: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Epoch 1/3                                            

 1/49 ━━━━━━━━━━━━━━━━━━━━ 1:02 1s/step - loss: 33.8609 - root_mean_squared_error: 5.8190
11/49 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 33.4827 - root_mean_squared_error: 5.7861 
22/49 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 32.0021 - root_mean_squared_error: 5.6551
30/49 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 30.9029 - root_mean_squared_error: 5.5550
37/49 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 29.9636 - root_mean_squared_error: 5.4675
47/49 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 28.6765 - root_mean_squared_error: 5.3446
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 28.3113 - root_mean_squared_error: 5.3091 - val_loss: 12.2222 - val_root_mean_squared_error: 3.4960

Epoch 2/3                                            

 1/49 ━━━━━━━━━━━━━━━━━━━━ 2s 62ms/step - loss: 10.8313 - root_mean_squared_error: 3.2911
11/49 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 10.8589 - root_mean_squared_error: 3.2951 
23/49 ━━━━━━━━━━━━━━━━━━━